# SEMANA 02
### MARYSOL CANTARERO VELAZQUEZ DEL VALLE

In [3]:
# Importando Paquetes
import pandas as pd
import numpy as np
from sklearn.neighbors.kde import KernelDensity
import matplotlib.pyplot as plt
from matplotlib.pyplot import plot
import scipy.stats as st
import statsmodels.datasets
import plotly.plotly as py
from plotly import tools
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
%matplotlib inline

In [135]:
# DATOS VALUACIÓN.
val = pd.read_excel('datos_ejemplo_1.xlsx')
val = info.fillna(0)
val = val['Valuacion']
val.head()

0    9.827297e+07
1    7.342905e+07
2    1.261004e+08
3    7.600080e+07
4    1.580456e+08
Name: Valuacion, dtype: float64

## Procesos a desarrollar
#### Requerimiento 1 Estimar función de densidad de probabilidad por método paramétrico

In [8]:
# función de densidad de probabilidad por método paramétrico
def best_fit_distribution(data, bins=200):    
    """Model data by finding best fit distribution to data"""
    '''O. Jaramillo Zuloaga'''
    y, x = np.histogram(data, bins=bins, density=True)
    x = (x + np.roll(x, -1))[:-1] / 2.0
    
    from scipy import stats as st
    DISTRIBUTIONS = [        
        st.alpha,st.anglit,st.arcsine,st.beta,st.betaprime,st.bradford,st.burr,st.cauchy,st.chi,st.chi2,st.cosine,
        st.dgamma,st.dweibull,st.erlang,st.expon,st.exponnorm,st.exponweib,st.exponpow,st.f,st.fatiguelife,st.fisk,
        st.foldcauchy,st.foldnorm,st.frechet_r,st.frechet_l,st.genlogistic,st.genpareto,st.gennorm,st.genexpon,
        st.genextreme,st.gausshyper,st.gamma,st.gengamma,st.genhalflogistic,st.gilbrat,st.gompertz,st.gumbel_r,
        st.gumbel_l,st.halfcauchy,st.halflogistic,st.halfnorm,st.halfgennorm,st.hypsecant,st.invgamma,st.invgauss,
        st.invweibull,st.johnsonsb,st.johnsonsu,st.ksone,st.kstwobign,st.laplace,st.levy,st.levy_l,st.levy_stable,
        st.logistic,st.loggamma,st.loglaplace,st.lognorm,st.lomax,st.maxwell,st.mielke,st.nakagami,st.ncx2,st.ncf,
        st.nct,st.norm,st.pareto,st.pearson3,st.powerlaw,st.powerlognorm,st.powernorm,st.rdist,st.reciprocal,
        st.rayleigh,st.rice,st.recipinvgauss,st.semicircular,st.t,st.triang,st.truncexpon,st.truncnorm,st.tukeylambda,
        st.uniform,st.vonmises,st.vonmises_line,st.wald,st.weibull_min,st.weibull_max,st.wrapcauchy
    ]
    
    best_distribution = st.norm
    best_params = (0.0, 1.0)
    best_sse = np.inf    
    for distribution in DISTRIBUTIONS:
        import warnings        
        try:                
            warnings.filterwarnings('ignore')                
            params = distribution.fit(data)
            arg = params[:-2]
            loc = params[-2]
            scale = params[-1]
            pdf = distribution.pdf(x, loc=loc, scale=scale, *arg)
            sse = np.sum(np.power(y - pdf, 2.0))                                
            if best_sse > sse:
                best_distribution = distribution
                best_params = params
                best_sse = sse
        except Exception:
            pass 
    return (best_distribution.name, best_params)

In [115]:
best_dist = best_fit_distribution(val)
best_dist

('vonmises_line', (1.4825214288992998, 99287930.75471371, 30669588.094487313))

In [138]:
# DISTRIBUCIÓN VONMISES LINE
dist = st.vonmises_line
args = dist.fit(val)
st.kstest(val, dist.cdf, args)

KstestResult(statistic=0.056594814705668184, pvalue=0.9090481974982458)

#### Requerimiento 2 Estimar función de densidad de probabilidad por método no paramétrico

In [19]:
kde = st.gaussian_kde(val,.1)

## Gráficas a desarrollar
#### Requerimiento 1 Graficar histograma de datos.

In [105]:
#CON PLOTLY
data= [go.Histogram(x=val, nbinsx= 30, name='Valuación')]
layout = go.Layout(title='HISTOGRAMA DE DATOS', xaxis=dict(title='Valuación'),yaxis=dict(title='Iteraciones'))
fig = go.Figure(data=data,layout=layout)
iplot(fig, filename = 'Hist')

#### Requerimiento 2 Graficar densidad de probabilidad ajustada superpuesta de histograma de datos

In [107]:
# MÉTODO PARAMÉTRICO
data1 = go.Scatter(x=x, y= (distvl * len(val) * val.max() / 30), mode= 'markers', name= 'Función vonmises line')
data2 = go.Histogram(x=val, nbinsx= 30, name='Valuación')
data= [data1, data2]
layout = go.Layout(title='Comparación con distribución obtenida', xaxis=dict(title='Iteración'),yaxis=dict(title='Valuación'))
fig = go.Figure(data=data,layout=layout)
iplot(fig, filename = 'Hist')

In [121]:
# MÉTODO NO PARAMÉTRICO
data1 = go.Scatter(x=x, y= (kde(x) * len(val) * val.max() / 30), mode= 'markers', name= 'Función no paramétrica(KDE)')
data2 = go.Histogram(x=val, nbinsx= 30, name='Valuación')
data= [data1, data2]
layout = go.Layout(title='Comparación con distribución obtenida', xaxis=dict(title='Iteraciones'),yaxis=dict(title='Valuación'))
fig = go.Figure(data=data,layout=layout)
iplot(fig, filename = 'Hist')

#### Requerimiento 3 Gráfica de linea para serie de tiempo y lineas verticales.

In [59]:
# PENDIENTE.